In [1]:
import numpy as np
import copy
from scipy.stats import multivariate_normal
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt

In [2]:
# # create data set
# n = 400
# _mus = np.array([[0,4], [-2,0]])
# _sigmas = np.array([[[3, 0], [0, 0.5]], [[1,0],[0,2]]])
# _pis = np.array([0.6, 0.4])
# X = np.concatenate([np.random.multivariate_normal(mu, sigma, int(pi*n))
#                     for pi, mu, sigma in zip(_pis, _mus, _sigmas)])

In [3]:
n=200
#number of samples per class: n
k=2
#number of classes: k
d=20
#dimensionality of samples: d
mus=np.random.random((k,d))*20-10 
#mean of the multivariate normal distribution | mus[i,j] is the mean for the ith cluster at the jth dimension
sigma=[np.eye(d) for i in range(k)]
#standard deviations are all assumed to be one. As the covariance matrix is identity the dimensions can be sampled independently
Pc=np.ones(k)/k
#setting priors: Pc

#Data Generation
X=[]
for i in range(n):
    for j in range(k):
#for each class generate n tuples of size d, taken from the distribution N(mus[j],1)
        point=[mean+np.random.randn() for mean in mus[j]]
        X.append(point)
#we have our dataset, where each point is a d+1 dimensional tuple where the last position represents its class 

X = np.asarray(X)

In [4]:
def w_mat(i,j):
    a = multivariate_normal.pdf(\
            X[index_matrix[i][j][0]],\
            mean=new_mus[index_matrix[i][j][1]],\
            cov=new_cov[index_matrix[i][j][1]],\
            allow_singular=True\
        )
    b = new_priors[index_matrix[i][j][1]]
    return a*b

def tabulate(x, y, f):
   """Return a table of f(x, y)."""
   #* is to unpack the two arrays which results after meshing
   return np.vectorize(f)( * np.meshgrid(x, y))

index_matrix = np.empty((X.shape[0],k),dtype=(int,2))
for i in range(index_matrix.shape[0]):
    for j in range(index_matrix.shape[1]):
        index_matrix[i][j] = (i,j)

def covid(n,m):
    co_vid = np.zeros((X.shape[1],X.shape[1]),dtype=float)
    for j in range(n):
        co_vid += W[m][j] * (((X[j] - new_mus[m]).T) @ (X[j] - new_mus[m]))
    return co_vid

In [5]:
d = X.shape[1]
n = X.shape[0]
# old_mus = np.array([np.random.rand(d) for _ in range(k)])
# random_rows = np.random.choice(X.shape[0], size=k, replace=False)
# old_mus = copy.deepcopy(X[random_rows, :])
old_mus = np.random.random((k,d))*20-10 
# old_mus = np.random.random((k,d))*np.max(X)-np.min(X)
new_mus = copy.deepcopy(old_mus)
print('original',mus)
print("initial", new_mus)

old_cov = np.asarray([np.eye(d) for _ in range(k)])
new_cov = copy.deepcopy(old_cov)

old_priors = np.full((k),1/k)
new_priors = copy.deepcopy(old_priors)

eps = 1e-7
t = 0
obt_eps = 1
while (obt_eps > eps) or (t==0):
    t += 1

    #unnormalised
    W_temp = copy.deepcopy(tabulate(list(range(X.shape[0])),list(range(k)),w_mat))
    temp = copy.deepcopy(W_temp)
    #normalised
    W = copy.deepcopy(temp/temp.sum(axis=0))
    # W = copy.deepcopy(W_temp)

    #get sum of W for each cluster
    temp = copy.deepcopy(W)
    sum_w = copy.deepcopy(temp.sum(axis=1))


    old_mus = copy.deepcopy(new_mus)
    #unnormalised
    new_mus_temp = copy.deepcopy(W @ X)
    temp = copy.deepcopy(new_mus_temp)
    #normalised
    new_mus = copy.deepcopy(temp/sum_w[:,None])
    # print(t,new_mus)

    old_cov = copy.deepcopy(new_cov)
    new_cov_temp = copy.deepcopy([covid(n,a) for a in range(k)])
    temp = copy.deepcopy(new_cov_temp)
    new_cov = copy.deepcopy([temp[a]/sum_w[a] for a in range(k)])

    old_priors = copy.deepcopy(new_priors)
    #normalised
    new_priors = copy.deepcopy(sum_w/n)
    obt_eps = np.sum([np.linalg.norm(new_mus[a]-old_mus[a]) for a in range(k)])
    # print("eps2",meh)

print("final", t,mus)


original [[ 2.48551134 -3.11577157 -3.86232128  0.09013514  7.48667121 -0.29942169
  -8.55110285 -3.84079984 -3.00463076 -4.20019479 -2.32425397  8.17727058
  -8.01212263 -4.41827962  4.19563984 -0.42784081 -1.28181521 -5.68588478
   5.96299376 -0.47788967]
 [-5.53927773  7.43367431 -6.67467411  0.88185634  6.27955169 -6.1171508
   6.28228708 -8.90140235 -9.65522301  7.49040436  6.85132198  6.89252763
  -8.38358974 -0.10613506 -7.80573055 -8.13119728 -1.29244227  8.68585525
  -3.44049265  2.35717638]]
initial [[ 6.29836674  1.58051253 -4.2736695   1.85917884  0.39321621 -2.61410007
  -4.98947431  3.50703816  7.37790564  4.61367026  3.999376    3.80095117
   9.85225631 -0.92653938  7.78041323  2.13287119 -6.11840291 -7.47451952
  -1.80568331 -1.03626597]
 [ 9.91811098 -2.86785996 -6.07642927 -1.65462822 -6.57096172 -1.37236243
   7.45416108 -5.01265119 -5.03279901 -0.58166157  2.19850875  0.60011416
  -7.10930212 -8.39841971  3.84416723 -1.43075613  2.53009874  1.71678461
   5.15849571 

In [66]:
# print('Original means of the data for class 1 are: {} and the predicted means are {}'.format(mus[0],means[0]))
# print('Original means of the data for class 2 are: {} and the predicted means are {}'.format(mus[1],means[1]))
# print('The Original covariance matrix is an Identity matrix for both the classes.')
# print('Predicted covariance matrices of the data for class 1 is: \n {} \n and for class 2 is \n {}'.format(sigmas[0],sigmas[1]))
